# Импорты

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt 
plt.style.use('ggplot')
import seaborn as sns 
sns.set(rc={'figure.figsize':(15, 5)})
import sys
import os
sys.path.append('/home/jovyan') 
from connectors import read_sql_query
from tqdm import tqdm

import sqlalchemy as sa
from scipy.stats import ttest_ind
from datetime import datetime
from typing import Tuple, List, Optional, Dict
import statsmodels.api as sm

## Функции

In [2]:
def get_query_clickhouse(q):
    ch_host = 'rc1b-t40nsk085ch73kmt.mdb.yandexcloud.net'
    ch_cert = '/home/jovyan/CA.pem'
    ch_port = '9440'
    ch_db   = 'analytics'
    ch_user = 'anastasiya_a_kosheleva'
    ch_pass = '092X4oRevXKWBAmB6ZfQBQlV5dNUczTxboSiisfY'

    engine = sa.create_engine(
        f'clickhouse+native://{ch_user}:{ch_pass}@{ch_host}:{ch_port}/{ch_db}?secure=True&ca_certs={ch_cert}'
    )

    return pd.read_sql_query(q, con=engine)

In [3]:
"""
Здесь описаны функции, которыми мы и считаем статтесты. 
Логика вся очень простая и вы всегда можете прочитать докстрингу.
"""

    
def calculate_pvalue(test: str, control: str, 
                     metric: str, group_id: str,
                     df_initial: pd.DataFrame, 
                     metric_type: Optional[str] = 'proportion') -> Tuple[float, float, float]:

    if metric_type == 'ratio':
        df = df_initial[[group_id, 'anonymous_id', metric[0], metric[1]]].dropna()
    else:
        df = df_initial[[group_id, 'anonymous_id', metric]].dropna()
        
    if metric_type == 'average' or metric_type == 'ratio':
        equal_var = False
    else:
        equal_var = True

    if metric_type == 'ratio':
        c_num_values = df[df[group_id] == control][metric[0]].values
        c_denom_values = df[df[group_id] == control][metric[1]].values
        
        t_num_values = df[df[group_id] == test][metric[0]].values
        t_denom_values = df[df[group_id] == test][metric[1]].values
        
        if c_denom_values.sum() == 0:
            print(metric)
        control_mean = c_num_values.sum() / c_denom_values.sum()
        test_mean = t_num_values.sum() / t_denom_values.sum()

        c_values = c_num_values - c_denom_values * control_mean
        t_values = t_num_values - t_denom_values * control_mean
    else:
        c_values = df[df[group_id] == control][metric].values
        t_values = df[df[group_id] == test][metric].values
        
        control_mean = c_values.mean()
        test_mean = t_values.mean()

    return ttest_ind(c_values, t_values, equal_var=equal_var).pvalue, control_mean, test_mean
        
        
def calculate_metrics(test: str, control: str, metrics: List[str],
                      group_id: str, df: pd.DataFrame, metric_type: str,
                      platform: Optional[str] = 'total') -> pd.DataFrame:
    """
    Вычисляет значение p-value
    - для proportion метрик (конверсионных)
        Используется стандартный t-test.
    - для average метрик (для денежных и других средних поюзерных метрик) 
        Используется Welch t-test с поправкой на неравенство дисперсий.
    - для ratio метрик(для глобальных средних метрик или ratio-метрик, например для среднего чека)
        Используется Welch t-test после линеаризации.
        Про линеаризацию тут https://instamart.atlassian.net/wiki/spaces/ANLT/pages/edit-v2/2061107789
    Параметры
    ----------
    test : str
        Название тестовой группы
    control : str
        Название контрольной группы
    metrics : list of str
        Список метрик, для которых нужно посчитать статистику
    group_id : str
        Название колонки, по которой определяется группа
    df : pd.Dataframe
        Датафрейм с данными, уровень группировки - айдишник пользователя. Примеры в запросе для денежных метрик.
    metric_type: str
        Тип метрики. Допустимые значения: 'average', 'ratio', 'proportion'
    """
    if platform == 'total':
        lst = []
        for metric in metrics:
            result =  calculate_pvalue(test, control, metric, group_id, df, metric_type=metric_type)
            lst.append([platform, metric, result[0], result[2], result[1]])
        df = pd.DataFrame(lst, columns=["platform", "metric", "p_value", "test_mean", "control_mean"])
        df["diff"] = df["test_mean"] - df["control_mean"]
        return df
    else:
        lst = []
        for metric in metrics:
            result =  calculate_pvalue(test, control, metric, group_id, df[df['platform'] == platform], metric_type=metric_type)
            lst.append([platform, metric, result[0], result[2], result[1]])
        df = pd.DataFrame(lst, columns=["platform", "metric", "p_value", "test_mean", "control_mean"])
        df["diff"] = df["test_mean"] - df["control_mean"]
        return df

In [4]:
"""
Здесь описаны функции, которыми мы и считаем статтесты. 
Логика вся очень простая и вы всегда можете прочитать докстрингу.
"""

    
def calculate_pvalue(test: str, control: str, 
                     metric: str, group_id: str,
                     df_initial: pd.DataFrame, 
                     metric_type: Optional[str] = 'proportion') -> Tuple[float, float, float]:

    if metric_type == 'ratio':
        df = df_initial[[group_id, 'anonymous_id', metric[0], metric[1]]].dropna()
    else:
        df = df_initial[[group_id, 'anonymous_id', metric]].dropna()
        
    if metric_type == 'average' or metric_type == 'ratio':
        equal_var = False
    else:
        equal_var = True

    if metric_type == 'ratio':
        c_num_values = df[df[group_id] == control][metric[0]].values
        c_denom_values = df[df[group_id] == control][metric[1]].values
        
        t_num_values = df[df[group_id] == test][metric[0]].values
        t_denom_values = df[df[group_id] == test][metric[1]].values
        
        if c_denom_values.sum() == 0:
            print(metric)
        control_mean = c_num_values.sum() / c_denom_values.sum()
        test_mean = t_num_values.sum() / t_denom_values.sum()

        c_values = c_num_values - c_denom_values * control_mean
        t_values = t_num_values - t_denom_values * control_mean
    else:
        c_values = df[df[group_id] == control][metric].values
        t_values = df[df[group_id] == test][metric].values
        
        control_mean = c_values.mean()
        test_mean = t_values.mean()

    return ttest_ind(c_values, t_values, equal_var=equal_var).pvalue, control_mean, test_mean
        
        
def calculate_metrics(test: str, control: str, metrics: List[str],
                      group_id: str, df: pd.DataFrame, metric_type: str,
                      platform: Optional[str] = 'total') -> pd.DataFrame:
    """
    Вычисляет значение p-value
    - для proportion метрик (конверсионных)
        Используется стандартный t-test.
    - для average метрик (для денежных и других средних поюзерных метрик) 
        Используется Welch t-test с поправкой на неравенство дисперсий.
    - для ratio метрик(для глобальных средних метрик или ratio-метрик, например для среднего чека)
        Используется Welch t-test после линеаризации.
        Про линеаризацию тут https://instamart.atlassian.net/wiki/spaces/ANLT/pages/edit-v2/2061107789
    Параметры
    ----------
    test : str
        Название тестовой группы
    control : str
        Название контрольной группы
    metrics : list of str
        Список метрик, для которых нужно посчитать статистику
    group_id : str
        Название колонки, по которой определяется группа
    df : pd.Dataframe
        Датафрейм с данными, уровень группировки - айдишник пользователя. Примеры в запросе для денежных метрик.
    metric_type: str
        Тип метрики. Допустимые значения: 'average', 'ratio', 'proportion'
    """
    if platform == 'total':
        lst = []
        for metric in metrics:
            result =  calculate_pvalue(test, control, metric, group_id, df, metric_type=metric_type)
            lst.append([platform, metric, result[0], result[2], result[1]])
        df = pd.DataFrame(lst, columns=["platform", "metric", "p_value", "test_mean", "control_mean"])
        df["diff"] = df["test_mean"] - df["control_mean"]
        return df
    else:
        lst = []
        for metric in metrics:
            result =  calculate_pvalue(test, control, metric, group_id, df[df['platform'] == platform], metric_type=metric_type)
            lst.append([platform, metric, result[0], result[2], result[1]])
        df = pd.DataFrame(lst, columns=["platform", "metric", "p_value", "test_mean", "control_mean"])
        df["diff"] = df["test_mean"] - df["control_mean"]
        return df


def get_exp_info(exp_id: str) -> Tuple[datetime.date, datetime.date]:
    """
    Выводит мета-информацию по тесту.
    Параметры
    ----------
    exp_id : str
        id теста из A/B-платформы
    
    Возвращает
    -------
    Выводит в аутпут метку теста и названия групп
    Возвращает:
    start_date : str
        Дата начала теста
    end_date : str
        Дата начала теста
    """

    q = f'''
        select 
            at.id as exp_id, 
            at.label as label,
            toDate(at.started_at) as start_date,
            if(
                toDate(at.ended_at) is null,
                today(),
                toDate(at.ended_at) - interval 1 day
            ) as end_date,
            atg.name as name,
            atg.is_reference as is_reference
        from 
            ods.bs_ab_admin__ab_tests at
        inner join 
            ods.bs_ab_admin__ab_test_groups atg on atg.test_id = at.id
        where true
            and toString(at.id) = '{exp_id}'
            and at.dwh_dt = (select max(dwh_dt) from ods.bs_ab_admin__ab_tests)
        '''

    test_info = get_query_clickhouse(q)

    exp_label = test_info['label'][0]
    start_date = test_info['start_date'][0]
    end_date = test_info['end_date'][0]
    
    print(f'Метка теста - {exp_label}')

    groups_dict = test_info[['name', 'is_reference']].T.to_dict('list')
    for group in groups_dict.values():
        if group[1] == 1:
            print(f'Контрольная группа - {group[0]}')
        else:
            print(f'Тестовая группа - {group[0]}')
    return start_date, end_date


def compute_metrics(test_group: str,
                    control_group: str,
                    metrics: Dict[str, List],
                    df: pd.DataFrame) -> pd.DataFrame:
    result = pd.DataFrame(columns = ['platform', 'metric', 'p_value',
                                     'test_mean', 'control_mean', 'diff'])
    
    platforms_list = list(df['platform'].unique())
    if len(platforms_list) > 1:
        platforms_list.append('total')
    
    for platform in platforms_list:
        average = calculate_metrics(
            test_group, control_group, metrics["average"], 'group', df, "average", platform
        )
        proportion = calculate_metrics(
            test_group, control_group, metrics["proportion"], 'group', df, "proportion", platform
        )
        ratio = calculate_metrics(
            test_group, control_group, metrics["ratio"], 'group', df, "ratio", platform
        )
        result = pd.concat([result, average, proportion, ratio]).reset_index(drop=True)
    return result


common_metrics = {
    "average": ['gmv_per_user', 'gmv_per_user_express', 'gmv_per_user_planned',
                'gmv_per_user_pickup', 'gross_profit_per_user', 'gross_profit_per_user_express',
                'gross_profit_per_user_planned', 'gross_profit_per_user_pickup', 'orders', 
                'express_orders', 'planned_orders', 'pickup_orders'],
    "proportion": ['conversion', 'express_conversion', 'planned_conversion', 'pickup_conversion',
                   'conv_to_second_order', 'conv_to_repeated_visit', 'ret_14d'], 
    "ratio": [('gmv_per_user', 'orders'), ('gmv_per_user_express', 'express_orders'), 
              ('gmv_per_user_planned', 'planned_orders'), ('gmv_per_user_pickup', 'pickup_orders')]
}


def drop_outliers(df: pd.DataFrame) -> pd.DataFrame:

    outliers_df = df.groupby("anonymous_id", as_index=False).agg(
        gmv_per_user_tmp=("gmv_per_user", "sum"),
        gmv_per_user_planned_tmp=("gmv_per_user_planned", "sum"),
        gmv_per_user_express_tmp=("gmv_per_user_express", "sum"),
        orders_tmp=("orders", "sum"),
        orders_planned_tmp=("planned_orders", "sum"),
        orders_express_tmp=("express_orders", "sum")
    )

    outliers_df["avg_check"] = np.where(
        outliers_df["orders_tmp"] != 0, outliers_df["gmv_per_user_tmp"] / outliers_df["orders_tmp"], 
        0
    )
    
    outliers_df["avg_check_planned"] = np.where(
        outliers_df["orders_planned_tmp"] != 0,
        outliers_df["gmv_per_user_planned_tmp"] / outliers_df["orders_planned_tmp"],
        0,
    )

    outliers_df["avg_check_express"] = np.where(
        outliers_df["orders_express_tmp"] != 0,
        outliers_df["gmv_per_user_express_tmp"] / outliers_df["orders_express_tmp"],
        0,
    )

    gmv = outliers_df[outliers_df["gmv_per_user_tmp"] > 0]
    gmv_mean = gmv["gmv_per_user_tmp"].mean()
    gmv_std = gmv["gmv_per_user_tmp"].std()
    
    gmv_planned = outliers_df[outliers_df["gmv_per_user_planned_tmp"] > 0]
    gmv_planned_mean = gmv_planned["gmv_per_user_planned_tmp"].mean()
    gmv_planned_std = gmv_planned["gmv_per_user_planned_tmp"].std()

    gmv_express = outliers_df[outliers_df["gmv_per_user_express_tmp"] > 0]
    gmv_express_mean = gmv_express["gmv_per_user_express_tmp"].mean()
    gmv_express_std = gmv_express["gmv_per_user_express_tmp"].std()

    orders = outliers_df[outliers_df["orders_tmp"] > 0]
    orders_mean = orders["orders_tmp"].mean()
    orders_std = orders["orders_tmp"].std()
    
    orders_planned = outliers_df[outliers_df["orders_planned_tmp"] > 0]
    orders_planned_mean = orders_planned["orders_planned_tmp"].mean()
    orders_planned_std = orders_planned["orders_planned_tmp"].std()
    
    orders_express = outliers_df[outliers_df["orders_express_tmp"] > 0]
    orders_express_mean = orders_express["orders_express_tmp"].mean()
    orders_express_std = orders_express["orders_express_tmp"].std()

    avg_check = outliers_df[outliers_df["avg_check"] > 0]
    avg_check_mean = avg_check["avg_check"].mean()
    avg_check_std = avg_check["avg_check"].std()

    avg_check_planned = outliers_df[outliers_df["avg_check_planned"] > 0]
    avg_check_planned_mean = avg_check_planned["avg_check_planned"].mean()
    avg_check_planned_std = avg_check_planned["avg_check_planned"].std()

    avg_check_express = outliers_df[outliers_df["avg_check_express"] > 0]
    avg_check_express_mean = avg_check_express["avg_check_express"].mean()
    avg_check_express_std = avg_check_express["avg_check_express"].std()

    try:
        outliers_df = outliers_df[
            (outliers_df["gmv_per_user_tmp"] >= (gmv_mean + (gmv_std * 9))) |
            (outliers_df["gmv_per_user_planned_tmp"] >= (gmv_planned_mean + (gmv_planned_std * 9))) |
            (outliers_df["gmv_per_user_express_tmp"] >= (gmv_express_mean + (gmv_express_std * 9))) |
            (outliers_df["orders_tmp"] >= (orders_mean + (orders_std * 9))) |
            (outliers_df["orders_planned_tmp"] >= (orders_planned_mean + (orders_planned_std * 9))) |
            (outliers_df["orders_express_tmp"] >= (orders_express_mean + (orders_express_std * 9))) |
            (outliers_df["avg_check"] >= (avg_check_mean + (avg_check_std * 9))) |
            (outliers_df["avg_check_planned"] >= (avg_check_planned_mean + (avg_check_planned_std * 9))) |
            (outliers_df["avg_check_express"] >= (avg_check_express_mean + (avg_check_express_std * 9)))
        ]
    except:
        outliers_df = outliers_df[outliers_df["anonymous_id"].isnull()]

    df = df[~df["anonymous_id"].isin(outliers_df["anonymous_id"])].reset_index(drop=True)
    return df


def style_rule(row):
    if row["p_value"] <= 0.01 and row["control_mean"] < row["test_mean"]:
        return ["background-color: #6ccd6c"] + [""]*(len(row)-1)
    elif row["p_value"] <= 0.05 and row["control_mean"] < row["test_mean"]:
        return ["background-color: palegreen"] + [""]*(len(row)-1)
    elif row["p_value"] <= 0.01 and row["control_mean"] > row["test_mean"]:
        return ["background-color: #fa6468"] + [""]*(len(row)-1)
    elif row["p_value"] <= 0.05 and row["control_mean"] > row["test_mean"]:
        return ["background-color: lightsalmon"] + [""]*(len(row)-1)
    else:
        return [""]*len(row)

## CUPED

In [5]:
def linearize(data: pd.DataFrame, 
              metric: str,
              numerator: str,
              denominator: str) -> pd.DataFrame:
    data_copy = data.copy()
    k = data_copy.loc[data_copy.treatment==0][numerator].sum() / \
            data_copy.loc[data_copy.treatment==0][denominator].sum()
    data_copy[metric] = data_copy[numerator] - k*data_copy[denominator]
    return data_copy.drop(columns=[denominator, numerator])

In [6]:
def calculate_cuped(pre_data: pd.DataFrame,
                    post_data: pd.DataFrame,
                    group_id: str,
                    test: str,
                    control: str,
                    metric: str,
                    metric_type: str,
                    numerator: Optional[str] = None,
                    denominator: Optional[str] = None) -> None:
    pre_tmp = pre_data.copy()
    post_tmp = post_data.copy()
    
    pre_tmp["treatment"] = np.where(pre_tmp[group_id] == test, 1, 0)
    post_tmp["treatment"] = np.where(post_tmp[group_id] == test, 1, 0)
    
    if metric_type == "ratio":
        lin_effect = post_tmp[post_tmp.treatment == 1][numerator].sum()/post_tmp[post_tmp.treatment == 1][denominator].sum() - post_tmp[post_tmp.treatment == 0][numerator].sum()/post_tmp[post_tmp.treatment == 0][denominator].sum()
        pre_tmp = linearize(pre_tmp, metric, numerator, denominator)
        post_tmp = linearize(post_tmp, metric, numerator, denominator)     
    
    pre_tmp = (pre_tmp.groupby("anonymous_id").agg({metric: "sum"})
                                                  .reset_index()
                                                  .rename({metric: "CUPED_X"}, axis=1))
            
    tmp = post_tmp.merge(pre_tmp, on="anonymous_id", how="left")
    
    # заполнение пропусков
    tmp["missing_CUPED"] = tmp["CUPED_X"].isna().astype(int)
    tmp["CUPED_X"] = tmp["CUPED_X"].fillna(0)
    
    # оценка y_cuped на контроле
    y_control = tmp.query("treatment==0")[metric]
    X_cov_control = tmp.query("treatment==0")[["CUPED_X", "missing_CUPED"]]
    y_hat = sm.OLS(y_control, X_cov_control).fit().predict(tmp[["CUPED_X", "missing_CUPED"]])
    tmp["y_cuped"] = tmp[metric].values - y_hat
    
    # cuped regression
    y_cuped = tmp["y_cuped"]
    X_cuped = tmp["treatment"]
    X_cuped = sm.add_constant(X_cuped)
    model_cuped = sm.OLS(y_cuped, X_cuped)
    results_cuped = model_cuped.fit()
    
    pvalue = results_cuped.pvalues[1]
        
    if metric_type != "ratio":
        effect = results_cuped.params[1]
        e = effect
        # print(f" pvalue for {metric}: {pvalue}. \n estimated effect for {metric}: {effect}")
    elif metric_type == "ratio": 
        e = lin_effect
        # print(f" pvalue for {metric}: {pvalue}. \n estimated effect for {metric}: {lin_effect}")
    return pd.DataFrame([(metric, metric_type, numerator, denominator, pvalue, e)], columns = ['metric','metric_type','numerator','denominator','pvalue','effect'])

In [7]:
def iter_cuped(metrics_dict,
               pre_test_df,
               test_df,
               test,
               control,
               group_id='group'
               ):
    out_df = pd.DataFrame()
    platforms = list(set(test_df.platform.unique().tolist() + ['total']))
    for platform in platforms:
        if platform == "total":
            pre_df = pre_test_df.copy()
            df = test_df.copy()
        else:
            pre_df = pre_test_df.query('platform==@platform').copy()
            df = test_df.query('platform==@platform').copy()
        for metric_type in metrics_dict:
            for metric in metrics_dict[metric_type]:
                if isinstance(metric,str):
                    tmp = calculate_cuped(
                            pre_data=pre_df,
                            post_data=df,
                            group_id=group_id,
                            test=test,
                            control=control,
                            metric=metric,
                            metric_type=metric_type)
                else:
                    tmp = calculate_cuped(
                            pre_data=pre_df,
                            post_data=df,
                            group_id=group_id,
                            test=test,
                            control=control,
                            metric=str(metric),
                            metric_type=metric_type,
                            numerator=metric[0],
                            denominator=metric[1])
                tmp['platform'] = platform
                out_df = pd.concat([out_df,tmp], ignore_index=True)
    return out_df

## Helpers

In [8]:
def create_sbx_table(sandbox_table_name,
                     suffix = '',
                     drop_if_exists=True):
    if suffix:
        sandbox_table_name = sandbox_table_name + '_' + suffix
    
    if not sandbox_table_name.startswith('sandbox'):
            raise Exception(f'"{sandbox_table_name}" is incorrect. sandbox_table_name must start with "sandbox"')
    
    if drop_if_exists:
        read_sql_query(f"""
        drop TABLE if exists {sandbox_table_name}
        """)
        print(f'Table {sandbox_table_name} dropped if existed')
        
    read_sql_query(f"""
        CREATE TABLE {sandbox_table_name}
        (
           `anonymous_id` String,
           `group` String,
           `platform` String,
           `tenant` String,
           `order_id` String,
           `events_dt` Date
        )
        ENGINE = MergeTree
        ORDER BY tuple()
        SETTINGS index_granularity = 8192
        """)
    print(f'Table {sandbox_table_name} created')
    return sandbox_table_name

# Константы

In [9]:
# Задаем параметры экспа, здесь с конкретным примером
exp_id = '5503fda5-7cff-427f-8b68-7ae08e887621'

In [10]:
start_date, end_date = get_exp_info(exp_id)

Метка теста - tech_oos_badge_test
Контрольная группа - control
Тестовая группа - oos_badge_group


In [11]:
control_group = 'control'
test_group = 'oos_badge_group'

In [12]:
preend_date = start_date - dt.timedelta(days=1)
prestart_date = preend_date - dt.timedelta(days=(end_date-start_date).days)
prestart_date, preend_date

(datetime.date(2024, 6, 13), datetime.date(2024, 7, 3))

In [13]:
oos_stores = read_sql_query(''' select df_oos.store_id as store_id, ret.id as retailer_id
from sandbox.stores_with_out_of_stock_model as df_oos
join int_stores as stores on df_oos.store_id = stores.id
join int_spree_retailers as ret on ret.id = stores.retailer_id
                            ''')
stores_list = oos_stores.store_id.to_list()

In [14]:
event_filter = f"""
'Category Viewed'
and store_id GLOBAL IN (select df_oos.store_id as store_id from sandbox.stores_with_out_of_stock_model as df_oos)
and params['category_id'] GLOBAL NOT IN (select distinct parent_id from ods.product_hub__taxon)
"""

ios_version_filter = f"""
(
  (
    platform = 'ios'
    and (app_version like '7%%' OR app_version like '8%%')
    and (
          (
            toUInt64OrZero(splitByChar('.', app_version)[1]) >= 7 
            and toUInt64OrZero(splitByChar('.', app_version)[2]) >= 16
          )
          or (toUInt64OrZero(app_version) >= 7160000)
          or toUInt64OrZero(splitByChar('.', app_version)[1]) >= 8
        )
  ) 
  OR platform != 'ios'
)
"""

sources_new_app = (
    "Order Completed",
    "Product Added",
    "Login",
    "Shop Selected",
    "Address Change Initiated",
    "Product Viewed",
    "Search Results Viewed",
    "Category Viewed",
    "Shop Selection Started",
    "Pickup Map Opened",
    "Retailer Selection Started",
    "Checkout Sber Spasibo Form Opened",
    "Map Pickup Shop Selected",
    "Checkout Button Clicked",
    "Checkout Loaded",
    "Main Page Viewed",
    "Checkout Delivery Slot Selection Started",
    "Checkout Delivery Slot Selected",
    "Cart Viewed",
    "Search Started",
    "Banner Viewed",
    "Recommendation Block View",
    "Product Category Viewed",
    "Catalogue Tab Clicked",
    "Onboarding Banner Viewed",
    "Order Info Viewed",
    "Product Sort Initiated",
    "Product Filter Initiated",
    "Product Sorted",
    "Address Selected",
    "Address Search Opened",
    "Address Suggest Clicked",
    "Redirected From SBOL",
    "Sberid Authorization Prompt Shown",
    "Map Pickup Button Clicked",
    "Age Confirmation Viewed",
    "Favorites Tab Clicked",
    "Address Change Clicked",
    "Retailer Selected"
)

sources_web = (
    "Order Completed",
    "Checkout Button Clicked",
    "Landing Viewed",
    "Product Added",
    "Main Page Viewed",
    "Suggester Viewed",
    "Main Page Viewed",
    "Shop Selected",
    "Product Viewed",
    "Recommendation Product View",
    "Search Results Viewed",
    "Category Viewed",
    "Shop Selection Started",
    "Pickup Map Opened",
    "Cart Viewed",
    "Order Merged",
    "Checkout Loaded",
    "Pre Replacements Product Selected",
    "Pre Replacements Product All Choosed",
    "Address Change Initiated",
    "Retailer Shipping Method Clicked",
    "Button Find Stores Clicked",
    "Favourites Tab Clicked",
    "Checkout Slot Selection Started",
    "Checkout Delivery Slot Selected",
    "Search Started",
    "Search Completed",
    "Search Suggest Clicked",
    "Suggester Viewed",
    "Banner Viewed",
    "Recommendation Block View",
    "Product Category Viewed",
    "Add To Cart Clicked",
    "Product Sort Initiated",
    "Product Sorted",
    "Cart Params Loaded",
    "Address Suggest Clicked",
    "Alcohol Pickup Button Clicked",
    "Only Pickup Popup Viewed",
    "Retailer Selected"
)

In [15]:
if event_filter:
    filter_events = f"""filter_events as (
    select
        anonymous_id,
        toDate(min(ts)) as event_dt
    from event.new_app
    where 1=1
        and event = {event_filter}
        and dwh_dt between start_date and end_date
        and toDate(ts) between start_date and end_date
        and {ios_version_filter}
    group by anonymous_id
    
    UNION ALL 

    select
        toString(anonymous_id) as anonymous_id,
        toDate(min(ts)) as event_dt
    from event.web
    where 1=1
        and event = {event_filter}
        and dwh_dt between start_date and end_date
        and toDate(ts) between start_date and end_date
    group by anonymous_id
    ),"""
    
    anon_in = """and anonymous_id global in 
    (
      select anonymous_id 
      from filter_events
    )"""
    
    join_filter = "inner join filter_events on ab_groups.anonymous_id = filter_events.anonymous_id"
    
    date_filter = "and filter_events.event_dt <= events_dt"
    
else:
    filter_events, anon_in, join_filter, date_filter = "", "", "", ""

# Базовые метрики

In [16]:
q = f"""
with 
    toDate('{start_date}') as start_date,
    toDate('{end_date}') as end_date,
    toString('{exp_id}') as exp_id,    

events as (
    select 
        toString(anonymous_id) as anonymous_id,
        toDate(ts) as dt,
        'web' as platform,
        ifNull(NullIf(tenant, ''), 'sbermarket') as tenant
    from event.web
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and not_bot
        and event in {sources_web}
    group by anonymous_id, dt, platform, tenant
    
    union all
    
    select 
        anonymous_id,
        toDate(ts) as dt,
        toString(platform) as platform,
        'sbermarket' as tenant
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and event in {sources_new_app}
        and {ios_version_filter}

    group by anonymous_id, dt, platform, tenant
),

{filter_events}

ab_groups as (
    select 
        anonymous_id,
        group,
        date_msk as dt
    from cdm.ab__groups__anon 
    where 1=1 
        and test_id = exp_id
        {anon_in}
),

client_id_x_order_id as (
    select distinct
        date_msk as date,
        uuid,
        anonymous_id,
        if(platform in ('desktop', 'mobile'), 'web', platform) as platform,
        tenant,
        order_id
    from 
        cdm.ab__client_id_x_order_id
    where true 
        and date_msk between start_date and end_date
),

ab_groups_x_events as (
    select distinct
        ab_groups.anonymous_id as anonymous_id,
        group,
        events.platform as platform,
        events.tenant as tenant,
        client_id_x_order_id.order_id as order_id,
        events.dt as events_dt
    from 
        ab_groups
        
        {join_filter}
    
        inner join events 
            on ab_groups.anonymous_id = events.anonymous_id 
        
        left join client_id_x_order_id
            on ab_groups.anonymous_id = client_id_x_order_id.anonymous_id
            and events.dt = client_id_x_order_id.date
            and events.platform = client_id_x_order_id.platform
            and events.tenant = client_id_x_order_id.tenant
            
    where true
      and ab_groups.dt <= events_dt
      {date_filter}
),

late_cancelled_orders as (
 select 
       stateful_id as order_id,
       minIf(toDate(created_at), next_state = 'complete') as completed_at,
       minIf(toDate(created_at), next_state = 'canceled') as canceled_at,
       if((canceled_at > completed_at + interval 2 day or canceled_at > end_date), 1, 0) as late_cancel_flg
  from (
  select stateful_type,
        created_at,
        next_state,
        stateful_id
    from analytics.int_spree_state_changes  
   where stateful_type = 'Spree::Order'
     and toDate(created_at) between start_date and end_date
     and next_state in ('complete', 'canceled')
    order by created_at) issc
 group by stateful_id
having completed_at is not null
   and late_cancel_flg
),

financial_measures as (
    select 
        uuid,
        order_number,
        toDate(completed_at) as completed_dt,
        max(shipped_at) as shipped_at,
        sumIf(1, type_store_delivery == 'planned') as planned_store_flg,
        sumIf(1, type_store_delivery == 'on_demand') as express_store_flg,
        sumIf(1, type_delivery=='asap') as express_flg,
        sumIf(1, type_delivery=='planned') as planned_flg,
        sumIf(1, type_delivery=='pickup') as pickup_flg,
        sumIf(gmv_service_fee_net_promo, type_store_delivery == 'on_demand') +
            sumIf(gmv_goods_net_promo, type_store_delivery == 'on_demand') as gmv_net_of_promo_express,
        sumIf(gmv_service_fee_net_promo, type_store_delivery == 'planned') +
            sumIf(gmv_goods_net_promo, type_store_delivery == 'planned') as gmv_net_of_promo_planned,
        sumIf(gmv_service_fee_net_promo, type_delivery = 'pickup') +
            sumIf(gmv_goods_net_promo, type_delivery = 'pickup') as gmv_net_of_promo_pickup,
        sum(gmv_service_fee_net_promo) + sum(gmv_goods_net_promo) as gmv_net_of_promo,
        sumIf(gross_profit, type_delivery = 'asap') as gross_profit_express,
        sumIf(gross_profit, type_delivery = 'planned') as gross_profit_planned,
        sumIf(gross_profit, type_delivery = 'pickup') as gross_profit_pickup,
        sumIf(gross_profit, type_store_delivery = 'on_demand') as gross_profit_express_store,
        sumIf(gross_profit, type_store_delivery = 'planned') as gross_profit_planned_store,
        sum(gross_profit) as gross_profit_full

    from gp_rep.rep__bi_shipment
    where 1=1
        and store_id IN {tuple(stores_list)}
        and toDate(completed_at) between start_date and end_date
        and user_id is not null
        and order_state in ('complete', 'canceled')
        and b2b_order_company_flg != 1
        and b2b_measure = 0
        and order_id not in (select order_id from late_cancelled_orders)
        and order_number not in 
        (
            select order_number
            from gp_rep.rep__bi_shipment
            where 1=1
                and toDate(completed_at) between start_date and end_date + interval 14 day
                and owner_type = 'ServiceAccount'
        )
    group by dictGet('analytics.spree_users_dict', 'uuid', toUInt64(user_id)) as uuid, order_number, completed_at
    )

select 
    group,
    anonymous_id,
    platform,
    tenant,
    uniqExactIf(order_id, uuid != '' and completed_dt = events_dt) as orders, -- это значение может быть использовано как знаменатель для ratio метрики среднего чека
    uniqExactIf(order_id, uuid != '' and express_flg > 0 and completed_dt = events_dt) as express_orders,
    uniqExactIf(order_id, uuid != '' and planned_flg > 0 and completed_dt = events_dt) as planned_orders,
    uniqExactIf(order_id, uuid != '' and express_store_flg > 0 and completed_dt = events_dt) as express_store_orders,
    uniqExactIf(order_id, uuid != '' and planned_store_flg > 0 and completed_dt = events_dt) as planned_store_orders,
    uniqExactIf(order_id, uuid != '' and pickup_flg > 0 and completed_dt = events_dt) as pickup_orders,
    
    max(if(uuid != '' and completed_dt = events_dt, 1, 0)) as conversion,
    max(if(uuid != '' and express_flg > 0 and completed_dt = events_dt, 1, 0)) as express_conversion,
    max(if(uuid != '' and planned_flg > 0 and completed_dt = events_dt, 1, 0)) as planned_conversion,
    max(if(uuid != '' and pickup_flg > 0 and completed_dt = events_dt, 1, 0)) as pickup_conversion,

    sumIf(gmv_net_of_promo, completed_dt = events_dt) as gmv_per_user, -- это значение может быть использовано как числитель для ratio метрики среднего чека
    sumIf(gmv_net_of_promo_express, completed_dt = events_dt) as gmv_per_user_express,
    sumIf(gmv_net_of_promo_planned, planned_flg > 0 and completed_dt = events_dt) as gmv_per_user_planned,
    sumIf(gmv_net_of_promo_pickup, pickup_flg > 0 and completed_dt = events_dt) as gmv_per_user_pickup,
    
    sumIf(gross_profit_full, completed_dt = events_dt) as gross_profit_per_user,
    sumIf(gross_profit_express, completed_dt = events_dt) as gross_profit_per_user_express,
    sumIf(gross_profit_planned, completed_dt = events_dt) as gross_profit_per_user_planned,
    sumIf(gross_profit_pickup, completed_dt = events_dt) as gross_profit_per_user_pickup,
    sumIf(gross_profit_express_store, express_store_flg > 0 and completed_dt = events_dt) as gross_profit_per_user_express_store,
    sumIf(gross_profit_planned_store, planned_store_flg > 0 and completed_dt = events_dt) as gross_profit_per_user_planned_store,

    minIf(completed_dt, completed_dt = events_dt) as first_order_date,
    if(
        first_order_date is null, 
        null, 
        arrayExists(
            elem -> assumeNotNull(elem <= toDate(end_date) and toDate(elem) > first_order_date), 
            groupArray(toDate(completed_dt))
        )
    ) as conv_to_second_order,
    
    if(
        first_order_date is null, 
        null, 
        arrayExists(
            elem -> assumeNotNull(elem <= toDate(end_date) and toDate(elem) > first_order_date), 
            groupArray(toDate(events_dt))
        )
    ) as conv_to_repeated_visit,    
    arrayExists(elem -> assumeNotNull(elem <= toDate(first_order_date + INTERVAL 14 DAY) and 
        elem > first_order_date), groupArray(toDate(shipped_at))) as ret_14d
from ab_groups_x_events
left join (select * from financial_measures 
            where completed_dt >= start_date and completed_dt <= end_date + interval 14 day) financial_measures
    on order_id = order_number
where tenant = 'sbermarket'
group by group, anonymous_id, platform, tenant
"""

In [17]:
df = get_query_clickhouse(q)
df.head()

,group,anonymous_id,platform,tenant,orders,express_orders,planned_orders,express_store_orders,planned_store_orders,pickup_orders,conversion,express_conversion,planned_conversion,pickup_conversion,gmv_per_user,gmv_per_user_express,gmv_per_user_planned,gmv_per_user_pickup,gross_profit_per_user,gross_profit_per_user_express,gross_profit_per_user_planned,gross_profit_per_user_pickup,gross_profit_per_user_express_store,gross_profit_per_user_planned_store,first_order_date,conv_to_second_order,conv_to_repeated_visit,ret_14d
0,oos_badge_group,42bc83a8-342b-4085-a439-877395f7358a,ios,sbermarket,4,4,0,4,0,0,1,1,0,0,2767.44,2767.44,0.00,0.0,544.40,544.40,0.00,0.0,544.40,0.00,2024-07-05,1.0,1.0,1
1,control,c9eedbb98db16713,android,sbermarket,1,0,1,0,1,0,1,0,1,0,4500.84,0.00,4500.84,0.0,376.87,0.00,376.87,0.0,0.00,376.87,2024-07-18,0.0,1.0,1
2,oos_badge_group,e80199ea-1f2b-4934-9949-ecb1d59b55ba,ios,sbermarket,1,1,0,1,0,0,1,1,0,0,1047.51,1047.51,0.00,0.0,142.93,142.93,0.00,0.0,142.93,0.00,2024-07-11,0.0,1.0,0
3,control,cc455a413c592d08,android,sbermarket,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,None,NaN,NaN,0
4,oos_badge_group,71fad0fa2e10d5ab,android,sbermarket,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,None,NaN,NaN,0


In [18]:
result_basic_metrics = compute_metrics(
    test_group=test_group, control_group=control_group, metrics=common_metrics, df=drop_outliers(df)
)
result_basic_metrics = result_basic_metrics.style.apply(style_rule, axis=1)

In [19]:
result_basic_metrics

,platform,metric,p_value,test_mean,control_mean,diff
0,ios,gmv_per_user,0.742454,1261.873482,1249.339510,12.533972
1,ios,gmv_per_user_express,0.759519,569.699053,563.974016,5.725037
2,ios,gmv_per_user_planned,0.625158,623.283703,608.427251,14.856453
3,ios,gmv_per_user_pickup,0.950518,31.382475,30.825418,0.557057
4,ios,gross_profit_per_user,0.893715,161.891802,162.567232,-0.675430
5,ios,gross_profit_per_user_express,0.597984,72.472310,73.853365,-1.381055
6,ios,gross_profit_per_user_planned,0.981732,86.681919,86.778402,-0.096483
7,ios,gross_profit_per_user_pickup,0.323626,2.283008,1.492152,0.790856
8,ios,orders,0.356185,0.682698,0.667913,0.014785
9,ios,express_orders,0.437824,0.434031,0.423891,0.010141


In [20]:
ab_groups = drop_outliers(df)[['group','anonymous_id','platform']]

# Кастомные метрики

### CR Product Rendered → Product Added

In [21]:
def get_cr_product_rendered_metrics_added(
                                 start_date, 
                                 end_date, 
                                 received_at_window=3):
    q = f"""
    with 
        toDate('{start_date}') as start_date,
        toDate('{end_date}') as end_date,
        toString('{exp_id}') as exp_id,    

events as (
    select 
        toString(anonymous_id) as anonymous_id,
        toDate(ts) as dt,
        'web' as platform,
        ifNull(NullIf(tenant, ''), 'sbermarket') as tenant
    from event.web
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and not_bot
        and event in {sources_web}
    group by anonymous_id, dt, platform, tenant
    
    union all
    
    select 
        anonymous_id,
        toDate(ts) as dt,
        toString(platform) as platform,
        'sbermarket' as tenant
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and event in {sources_new_app}
        and {ios_version_filter}

    group by anonymous_id, dt, platform, tenant
),

{filter_events}

ab_groups as (
    select 
        anonymous_id,
        group,
        date_msk as dt
    from cdm.ab__groups__anon 
    where 1=1 
        and test_id = exp_id
        {anon_in}
),

client_id_x_order_id as (
    select distinct
        date_msk as date,
        uuid,
        anonymous_id,
        if(platform in ('desktop', 'mobile'), 'web', platform) as platform,
        tenant,
        order_id
    from 
        cdm.ab__client_id_x_order_id
    where true 
        and date_msk between start_date and end_date
),

ab_groups_x_events as (
    select distinct
        ab_groups.anonymous_id as anonymous_id,
        group,
        events.platform as platform,
        events.tenant as tenant,
        client_id_x_order_id.order_id as order_id,
        events.dt as events_dt
    from 
        ab_groups
        
        {join_filter}
    
        inner join events 
            on ab_groups.anonymous_id = events.anonymous_id 
        
        left join client_id_x_order_id
            on ab_groups.anonymous_id = client_id_x_order_id.anonymous_id
            and events.dt = client_id_x_order_id.date
            and events.platform = client_id_x_order_id.platform
            and events.tenant = client_id_x_order_id.tenant
            
    where true
    and ab_groups.dt <= events_dt
    {date_filter}
),

    t1 as (

    select 
           anonymous_id,
           count(ts) as if_rendered
      from event.new_app
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Rendered'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
       and {ios_version_filter}
     group by anonymous_id

    UNION ALL 

    select 
           toString(anonymous_id) AS anonymous_id,
           count(ts) as if_rendered
      from event.web
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Rendered'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
     group by anonymous_id
    ),

    t2 as (

    select 
           anonymous_id,
           count(ts) as if_added
      from event.new_app
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Added'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
       and {ios_version_filter}
     group by anonymous_id

    UNION ALL 

    select 
           toString(anonymous_id) AS anonymous_id,
           count(ts) as if_added
      from event.web
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Added'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
     group by anonymous_id
    ),

    
    final_table as (
    select
        a.anonymous_id as anonymous_id,
        if_rendered,
        if_added
    from t1 a left join t2 b on a.anonymous_id = b.anonymous_id
    )
    select 
           ab_groups_x_events.events_dt AS dt,
           ab_groups_x_events.platform AS platform,
           ab_groups_x_events.anonymous_id AS anonymous_id,
           SUM(if_rendered) AS product_rendered,
           SUM(if_added) AS product_added
      from ab_groups_x_events a 
           join final_table b ON a.anonymous_id = b.anonymous_id
     GROUP BY 1,2,3

    SETTINGS enable_optimize_predicate_expression=0
    """
    return read_sql_query(q)

In [22]:
cr_product_rendered_metrics_added_data = get_cr_product_rendered_metrics_added(start_date, end_date)

In [23]:
cr_product_rendered_metrics_added_data

,dt,platform,anonymous_id,product_rendered,product_added
0,2024-07-18,android,f2f1180d553f3680,8,4
1,2024-07-21,android,78a9ae6106d59d94,1,0
2,2024-07-10,android,dc04e7265f211c47,56,5
3,2024-07-10,android,0745c7cf717b3789,4,0
4,2024-07-10,android,8706317bb14f096c,6,0
...,...,...,...,...,...
125039,2024-07-14,android,b0f9e8403d65af47,1,0
125040,2024-07-14,android,3bcd3b771a9e1026,12,0
125041,2024-07-23,android,4ba1919710a9f902,1,0
125042,2024-07-10,android,56e304fad030f1d6,48,0


In [24]:
cr_product_rendered_metrics_added_data = (
    cr_product_rendered_metrics_added_data
    .merge(ab_groups, on=['platform','anonymous_id'])
)

In [25]:
cr_product_rendered_metrics_added = \
{'average':[],
 'proportion': [],
 'ratio': [
    ("product_added", "product_rendered") 
 ]}

cr_product_rendered_metrics_added_result = compute_metrics(
    test_group=test_group, control_group=control_group, metrics=cr_product_rendered_metrics_added, \
      df=cr_product_rendered_metrics_added_data
)

In [26]:
cr_product_rendered_metrics_added_result = cr_product_rendered_metrics_added_result.style.apply(style_rule, axis=1)
cr_product_rendered_metrics_added_result

,platform,metric,p_value,test_mean,control_mean,diff
0,android,"('product_added', 'product_rendered')",0.736912,0.032398,0.032211,0.000187
1,ios,"('product_added', 'product_rendered')",0.390191,0.026610,0.027401,-0.000791
2,total,"('product_added', 'product_rendered')",0.750101,0.031070,0.031224,-0.000154


In [27]:
def get_cr_product_rendered_metrics_order(
                                 start_date, 
                                 end_date, 
                                 received_at_window=3):
    q = f"""
    with 
        toDate('{start_date}') as start_date,
        toDate('{end_date}') as end_date,
        toString('{exp_id}') as exp_id,    

events as (
    select 
        toString(anonymous_id) as anonymous_id,
        toDate(ts) as dt,
        'web' as platform,
        ifNull(NullIf(tenant, ''), 'sbermarket') as tenant
    from event.web
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and not_bot
        and event in {sources_web}
    group by anonymous_id, dt, platform, tenant
    
    union all
    
    select 
        anonymous_id,
        toDate(ts) as dt,
        toString(platform) as platform,
        'sbermarket' as tenant
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and event in {sources_new_app}
        and {ios_version_filter}

    group by anonymous_id, dt, platform, tenant
),

{filter_events}

ab_groups as (
    select 
        anonymous_id,
        group,
        date_msk as dt
    from cdm.ab__groups__anon 
    where 1=1 
        and test_id = exp_id
        {anon_in}
),

client_id_x_order_id as (
    select distinct
        date_msk as date,
        uuid,
        anonymous_id,
        if(platform in ('desktop', 'mobile'), 'web', platform) as platform,
        tenant,
        order_id
    from 
        cdm.ab__client_id_x_order_id
    where true 
        and date_msk between start_date and end_date
),

ab_groups_x_events as (
    select distinct
        ab_groups.anonymous_id as anonymous_id,
        group,
        events.platform as platform,
        events.tenant as tenant,
        client_id_x_order_id.order_id as order_id,
        events.dt as events_dt
    from 
        ab_groups
        
        {join_filter}
    
        inner join events 
            on ab_groups.anonymous_id = events.anonymous_id 
        
        left join client_id_x_order_id
            on ab_groups.anonymous_id = client_id_x_order_id.anonymous_id
            and events.dt = client_id_x_order_id.date
            and events.platform = client_id_x_order_id.platform
            and events.tenant = client_id_x_order_id.tenant
            
    where true
    and ab_groups.dt <= events_dt
    {date_filter}
),

    t1 as (

    select 
           anonymous_id,
           count(ts) as if_rendered
      from event.new_app
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Rendered'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
       and {ios_version_filter}
     group by anonymous_id

    UNION ALL 

    select 
           toString(anonymous_id) AS anonymous_id,
           count(ts) as if_rendered
      from event.web
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Rendered'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
     group by anonymous_id
    ),

    t2 as (

    select 
           anonymous_id,
           count(ts) as order_completed
      from event.new_app
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Order Completed'
       and {ios_version_filter}
     group by anonymous_id

    UNION ALL 

    select 
           toString(anonymous_id) AS anonymous_id,
           count(ts) as order_completed
      from event.web
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Order Completed'
     group by anonymous_id
    ),

    
    final_table as (
    select
        a.anonymous_id as anonymous_id,
        if_rendered,
        order_completed
    from t1 a left join t2 b on a.anonymous_id = b.anonymous_id
    )
    select 
           ab_groups_x_events.events_dt AS dt,
           ab_groups_x_events.platform AS platform,
           ab_groups_x_events.anonymous_id AS anonymous_id,
           SUM(if_rendered) AS product_rendered,
           SUM(order_completed) AS order_completed
      from ab_groups_x_events a 
           join final_table b ON a.anonymous_id = b.anonymous_id
     GROUP BY 1,2,3

    SETTINGS enable_optimize_predicate_expression=0
    """
    return read_sql_query(q)

In [28]:
cr_product_rendered_metrics_order_data = get_cr_product_rendered_metrics_order(start_date, end_date)

In [ ]:
cr_product_rendered_metrics_order_data = (
    cr_product_rendered_metrics_order_data
    .merge(ab_groups, on=['platform','anonymous_id'])
)

In [ ]:
cr_product_rendered_metrics_order = \
{'average':[],
 'proportion': [],
 'ratio': [
    ("order_completed", "product_rendered") 
 ]}

cr_product_rendered_metrics_order_result = compute_metrics(
    test_group=test_group, control_group=control_group, metrics=cr_product_rendered_metrics_order, \
      df=cr_product_rendered_metrics_order_data
)

In [ ]:
cr_product_rendered_metrics_order_result = cr_product_rendered_metrics_order_result.style.apply(style_rule, axis=1)
cr_product_rendered_metrics_order_result

,platform,metric,p_value,test_mean,control_mean,diff
0,ios,"('order_completed', 'product_rendered')",0.382622,0.112897,0.109572,0.003325
1,android,"('order_completed', 'product_rendered')",0.000000,0.111347,0.090373,0.020974
2,total,"('order_completed', 'product_rendered')",0.000000,0.111761,0.094935,0.016826


In [ ]:
def get_avg_count_a2c(
                                 start_date, 
                                 end_date, 
                                 received_at_window=3):
    q = f"""
    with 
        toDate('{start_date}') as start_date,
        toDate('{end_date}') as end_date,
        toString('{exp_id}') as exp_id,    

events as (
    select 
        toString(anonymous_id) as anonymous_id,
        toDate(ts) as dt,
        'web' as platform,
        ifNull(NullIf(tenant, ''), 'sbermarket') as tenant
    from event.web
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and not_bot
        and event in {sources_web}
    group by anonymous_id, dt, platform, tenant
    
    union all
    
    select 
        anonymous_id,
        toDate(ts) as dt,
        toString(platform) as platform,
        'sbermarket' as tenant
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and event in {sources_new_app}
        and {ios_version_filter}

    group by anonymous_id, dt, platform, tenant
),

{filter_events}

ab_groups as (
    select 
        anonymous_id,
        group,
        date_msk as dt
    from cdm.ab__groups__anon 
    where 1=1 
        and test_id = exp_id
        {anon_in}
),

client_id_x_order_id as (
    select distinct
        date_msk as date,
        uuid,
        anonymous_id,
        if(platform in ('desktop', 'mobile'), 'web', platform) as platform,
        tenant,
        order_id
    from 
        cdm.ab__client_id_x_order_id
    where true 
        and date_msk between start_date and end_date
),

ab_groups_x_events as (
    select distinct
        ab_groups.anonymous_id as anonymous_id,
        group,
        events.platform as platform,
        events.tenant as tenant,
        client_id_x_order_id.order_id as order_id,
        events.dt as events_dt
    from 
        ab_groups
        
        {join_filter}
    
        inner join events 
            on ab_groups.anonymous_id = events.anonymous_id 
        
        left join client_id_x_order_id
            on ab_groups.anonymous_id = client_id_x_order_id.anonymous_id
            and events.dt = client_id_x_order_id.date
            and events.platform = client_id_x_order_id.platform
            and events.tenant = client_id_x_order_id.tenant
            
    where true
    and ab_groups.dt <= events_dt
    {date_filter}
),

    t1 as (

    select 
           anonymous_id,
           count(ts) as if_added
      from event.new_app
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Added'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
       and {ios_version_filter}
     group by anonymous_id

    UNION ALL 

    select 
           toString(anonymous_id) AS anonymous_id,
           count(ts) as if_added
      from event.web
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Added'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
     group by anonymous_id
    )


    select 
           ab_groups_x_events.events_dt AS dt,
           ab_groups_x_events.platform AS platform,
           ab_groups_x_events.anonymous_id AS anonymous_id,
           SUM(if_added) AS avg_product_added
      from ab_groups_x_events a 
           join t1 b ON a.anonymous_id = b.anonymous_id
     GROUP BY 1,2,3

    SETTINGS enable_optimize_predicate_expression=0
    """
    return read_sql_query(q)

In [ ]:
avg_count_a2c_data = get_avg_count_a2c(start_date, end_date)

In [ ]:
avg_count_a2c_data = (
    avg_count_a2c_data
    .merge(ab_groups, on=['platform','anonymous_id'])
)

In [ ]:
avg_count_a2c_metrics = \
{'average':[  ("avg_product_added") ],
 'proportion': [],
 'ratio': []}

avg_count_a2c_result = compute_metrics(
    test_group=test_group, control_group=control_group, metrics=avg_count_a2c_metrics, \
      df=avg_count_a2c_data
)

In [ ]:
avg_count_a2c_result = avg_count_a2c_result.style.apply(style_rule, axis=1)
avg_count_a2c_result

,platform,metric,p_value,test_mean,control_mean,diff
0,android,avg_product_added,0.030926,4.135712,4.322758,-0.187047
1,ios,avg_product_added,0.874082,3.617093,3.600131,0.016962
2,total,avg_product_added,0.037137,4.002251,4.148894,-0.146643


In [ ]:
def get_rc_share_in_orders(
                                 start_date, 
                                 end_date, 
                                 received_at_window=3):
    q = f"""
    with 
        toDate('{start_date}') as start_date,
        toDate('{end_date}') as end_date,
        toString('{exp_id}') as exp_id,    

events as (
    select 
        toString(anonymous_id) as anonymous_id,
        toDate(ts) as dt,
        'web' as platform,
        ifNull(NullIf(tenant, ''), 'sbermarket') as tenant
    from event.web
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and not_bot
        and event in {sources_web}
    group by anonymous_id, dt, platform, tenant
    
    union all
    
    select 
        anonymous_id,
        toDate(ts) as dt,
        toString(platform) as platform,
        'sbermarket' as tenant
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and event in {sources_new_app}
        and {ios_version_filter}

    group by anonymous_id, dt, platform, tenant
),

{filter_events}

ab_groups as (
    select 
        anonymous_id,
        group,
        date_msk as dt
    from cdm.ab__groups__anon 
    where 1=1 
        and test_id = exp_id
        {anon_in}
),

client_id_x_order_id as (
    select distinct
        date_msk as date,
        uuid,
        anonymous_id,
        if(platform in ('desktop', 'mobile'), 'web', platform) as platform,
        tenant,
        order_id
    from 
        cdm.ab__client_id_x_order_id
    where true 
        and date_msk between start_date and end_date
),

ab_groups_x_events as (
    select distinct
        ab_groups.anonymous_id as anonymous_id,
        group,
        events.platform as platform,
        events.tenant as tenant,
        client_id_x_order_id.order_id as order_id,
        events.dt as events_dt
    from 
        ab_groups
        
        {join_filter}
    
        inner join events 
            on ab_groups.anonymous_id = events.anonymous_id 
        
        left join client_id_x_order_id
            on ab_groups.anonymous_id = client_id_x_order_id.anonymous_id
            and events.dt = client_id_x_order_id.date
            and events.platform = client_id_x_order_id.platform
            and events.tenant = client_id_x_order_id.tenant
            
    where true
    and ab_groups.dt <= events_dt
    {date_filter}
),

    t1 as (

    select 
           anonymous_id
      from event.new_app
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Added'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
       and {ios_version_filter}
     group by 1

    UNION ALL 

    select 
           toString(anonymous_id) AS anonymous_id
      from event.web
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Added'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (72, 52)
       and source in ('category', 'search', 'search_multiretail')
     group by 1
    ),
    
    t2 as (
    SELECT order_number,
       count(distinct case when li_deleted_at is not null then retailer_sku else null end) as rc_num,
       count(distinct retailer_sku) as rc_denom
  FROM line_items
    where 1 = 1
        and toDate(order_completed_at) between start_date and end_date
        and toDate(li_created_at) BETWEEN start_date-60 AND end_date
        AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
        AND shipment_state = 'shipped'
        AND li_created_at is not null
        AND not (li_deleted_at is not null and assembly_issue is null)
        AND sku not in (select sku from sandbox.damage_packages)
        AND retailer_sku <> '4000103813'
        AND order_number global in (select order_id from  ab_groups_x_events)
    group by 1
    )

    select 
           ab_groups_x_events.events_dt AS dt,
           ab_groups_x_events.platform AS platform,
           ab_groups_x_events.anonymous_id AS anonymous_id,
           SUM(rc_num) AS rc_num,
           SUM(rc_denom) AS rc_denom
      from ab_groups_x_events a 
           join t2 b ON a.order_id = toString(b.order_number)
     GROUP BY 1,2,3

    SETTINGS enable_optimize_predicate_expression=0
    """
    return read_sql_query(q)

In [ ]:
rc_share_in_orders_data = get_rc_share_in_orders(start_date, end_date)

In [ ]:
rc_share_in_orders_data = (
    rc_share_in_orders_data
    .merge(ab_groups, on=['platform','anonymous_id'])
)

In [ ]:
rc_share_in_orders_data_metrics = \
{'average':[  ("rc_num") ],
 'proportion': [],
 'ratio': [("rc_num", "rc_denom")]}

In [ ]:
rc_share_in_orders_data_result = compute_metrics(
    test_group=test_group, control_group=control_group, metrics=rc_share_in_orders_data_metrics, \
      df=rc_share_in_orders_data
)

In [ ]:
rc_share_in_orders_data_result = rc_share_in_orders_data_result.style.apply(style_rule, axis=1)
rc_share_in_orders_data_result

,platform,metric,p_value,test_mean,control_mean,diff
0,ios,rc_num,0.278993,0.655828,0.637506,0.018323
1,ios,"('rc_num', 'rc_denom')",0.174681,0.043635,0.042368,0.001267
2,android,rc_num,0.000000,0.795884,0.866454,-0.070570
3,android,"('rc_num', 'rc_denom')",0.000000,0.052466,0.056909,-0.004443
4,total,rc_num,0.000170,0.749482,0.789687,-0.040205
5,total,"('rc_num', 'rc_denom')",0.000015,0.049559,0.052071,-0.002513


In [ ]:
df_list = []
df_list.append(result_basic_metrics, cr_product_rendered_metrics_added_result, cr_product_rendered_metrics_order_result, avg_count_a2c_result, rc_share_in_orders_data_result)
final_result_table = pd.concat(df_list)

: 

In [ ]:
final_result_table.to_excel('oos_tovara_ostalos_malo_result.xlsx')